# Trabajo Práctico: Agente para automatizar la búsqueda de repuestos

- **Curso:** DUIA - 2025, Módulo 6
- **Integrantes:** David Burckhardt, Martin Vazquez Arispe, Martin Caballero.
- **Objetivo:** Implementar un sistema inteligente que automatice la búsqueda, ranking y pedido de repuestos para una empresa distribuidora.

---
##  Índice del Notebook

1. **Consigna del trabajo**
2. **Configuracion de API Keys**

---

## 1. Consigna: agente(s) para automatizar la búsqueda de repuestos
- Dada una solicitud de repuestos específicos para una empresa distribuidora, un
agente debe identificar las especificaciones de dichos repuestos (según un
catálogo), a fin de poder buscarlos.
- El agente busca en primer lugar en el inventario de la empresa, y en en caso de
no encontrarlos (puede ser que encuentre solo algunos de ellos), debe consultar
en catálogos de proveedores.
- El sistema extrae información de las opciones encontradas, y genera un ranking
de alternativas, priorizando: 
    - Repuestos internos (si están disponibles).
    - Proveedores externos según criterios de optimización (por ej. costo-beneficio).
- Para repuestos internos: 
    - Se genera una orden de retiro del inventario y se notifica al almacén para su preparación. 
- Para repuestos externos: 
    - se envía un email automatizado al proveedor seleccionado para formalizar el pedido.

- Finalmente, se agenda la fecha estimada de entrega y detalles del pedido en el
sistema de seguimiento.
- Pueden incluirse pasos de "human in the loop" para verificar resultados antes de
tomar acciones

## 2. Configuración de API Keys y variables de entorno

Cargamos la `GROQ_API_KEY` desde el archivo `.env` e inicializamos el cliente LLM.

**Nota:** Asegúrate de tener un archivo `.env` en el directorio raíz con:
```
GROQ_API_KEY=tu_clave_aqui
```

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

# Cargar variables de entorno
load_dotenv()

# Verificar que la API key está configurada
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY no encontrada en .env")

# Inicializar el LLM de Groq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    api_key=api_key
)

print("LLM de Groq inicializado correctamente")
print(f"   Modelo: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")


/opt/anaconda3/envs/DUIA/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


LLM de Groq inicializado correctamente
   Modelo: llama-3.3-70b-versatile
   Temperature: 0.1


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

with open('prompts/system_prompt.txt', 'r', encoding='utf-8') as f:
    SYSTEM_PROMPT = f.read()

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

# Create the chain
chain = prompt | llm

import langchain_core
print(f"Versión de LangChain-Core: {langchain_core.__version__}")

Versión de LangChain-Core: 1.0.7


In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, Optional
from pydantic import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver
import re

# Mongo DB

In [ ]:
from pymongo import MongoClient
import os

# Conectar a MongoDB Atlas
MONGO_URI = os.getenv("MONGO_URI")

if not MONGO_URI:
    raise ValueError("MONGO_URI no encontrada en .env")

client = MongoClient(MONGO_URI)
db = client.db_respuestos
collection = db.repuestos

print("✅ Conexión a MongoDB establecida")
print(f"   Base de datos: db_respuestos")
print(f"   Colección: repuestos")

{'_id': ObjectId('691fceeac7525a752e5db161'), 'id_repuesto': 'R-0001', 'repuesto_descripcion': 'Rodamiento rígido de bolas 6204 2RS', 'categoria': 'RODAMIENTO', 'marca': 'SKF', 'modelo': '6204-2RS', 'proveedor_tipo': 'EXTERNAL', 'proveedor_id': 'PRV-001', 'proveedor_nombre': 'Distribuidora Rodamax', 'proveedor_rating': 4, 'costo_unitario': 7.9, 'moneda': 'USD', 'stock_disponible': 100, 'lead_time_dias': 5, 'ubicacion_stock': 'Prov. Buenos Aires', 'cantidad_minima_pedido': 5, 'tiempo_vida_estimado_hrs': 10000, 'nota': 'Descuento por compras mayores a 50 unidades'}
{'_id': ObjectId('691fceeac7525a752e5db162'), 'id_repuesto': 'R-0001', 'repuesto_descripcion': 'Rodamiento rígido de bolas 6204 2RS', 'categoria': 'RODAMIENTO', 'marca': 'GENERICA', 'modelo': '6204-2RS', 'proveedor_tipo': 'EXTERNAL', 'proveedor_id': 'PRV-002', 'proveedor_nombre': 'Importadora GenBearing', 'proveedor_rating': 3, 'costo_unitario': 5.4, 'moneda': 'USD', 'stock_disponible': 300, 'lead_time_dias': 10, 'ubicacion_st

# Estado

In [5]:
class ValidationRequest(BaseModel):
    is_parts_request: bool = Field(
        default=False,
        description="True si la consulta es una solicitud de repuestos o piezas. False si es una pregunta general o spam."
    )
    message: str = Field(
        default="",
        description="Mensaje del agente. Si es un pedido de repuestos, debe incluir los pasos siguientes. Indicar al cliente que debe se deben realizar consultas sobre repuestos."
    )

#Definimos el esquema mejorado
class AgentState(TypedDict):
    validation_result: ValidationRequest
    messages: Annotated[list, add_messages]
    codigo_repuesto: Optional[str]  # Código del repuesto (R-XXXX)
    info_completa: bool  # Si tenemos toda la información necesaria
    resultados_internos: list[dict]  # Lista de resultados de repuestos internosq

# Nodo de validación de intención

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

CLASSIFIER_SYSTEM_PROMPT = (
    "Eres un clasificador de mensajes experto. Tu única tarea es determinar si el siguiente texto "
    "es una solicitud formal de repuestos, piezas, o componentes específicos. "
    "Responde SOLO con un objeto JSON que siga el esquema proporcionado"
    "\n\nINSTRUCCIÓN CLAVE: En el campo 'message', debes generar una respuesta concisa para el usuario. "
    "Si 'is_parts_request' es True, indica que el pedido será procesado. "
    "Si 'is_parts_request' es False, explica la razón del descarte (Ej: 'La consulta no es un pedido de repuestos')."
)

validation_prompt = ChatPromptTemplate.from_messages([
    ("system", CLASSIFIER_SYSTEM_PROMPT),
    ("placeholder", "{messages}")
])

parser = PydanticOutputParser(pydantic_object=ValidationRequest)

validation_prompt_con_instrucciones = ChatPromptTemplate.from_messages(
    validation_prompt.messages + [
        ("human", "{format_instructions}")
    ]
).partial(
    format_instructions=parser.get_format_instructions()
)

validation_chain = (
    validation_prompt_con_instrucciones 
    | llm 
    | parser
)

def classify_request(state: AgentState) -> AgentState:
    messages: list[BaseMessage] = state['messages']
    
    validation_result_object = validation_chain.invoke({"messages": messages})
    
    return {"validation_result": validation_result_object}

def set_val_message(state: AgentState) -> AgentState:
    return {"messages": [state['validation_result'].message]}

def route_classification(state: AgentState) -> str:
    print(f"Dentro de route: {state['validation_result'].is_parts_request}")
    if state['validation_result'].is_parts_request:
        return "continue"
    else:
        return "end"

In [7]:
#Defino funcion del nodo
def nodo_llm(state: AgentState) -> AgentState:
    messages = state["messages"]
    response = chain.invoke({"messages": messages})
    
    # Extraer código de repuesto si está presente en los mensajes
    codigo_repuesto = state.get("codigo_repuesto")
    
    # Buscar patrón R-XXXX en todos los mensajes del usuario
    for msg in messages:
        if isinstance(msg, HumanMessage):
            # Buscar patrón R-XXXX (ej: R-0001, R-0123)
            match = re.search(r'R-\d{4}', msg.content, re.IGNORECASE)
            if match:
                codigo_repuesto = match.group(0).upper()
                break
    
    # Determinar si tenemos información completa
    info_completa = codigo_repuesto is not None
    
    return {
        "messages": [response],
        "codigo_repuesto": codigo_repuesto,
        "info_completa": info_completa
    }

#Defino la funcion para decidir si continuamos o terminamos
def tiene_info_suficiente(state: AgentState) -> str:
    # Si tenemos el código del repuesto, terminamos
    if not(state.get("info_completa", False) and state.get("codigo_repuesto")):
        return "request_more_info"
    else:
        print(f"\n✅ Código de repuesto identificado: {state['codigo_repuesto']}")
        return "continue"

def search_internal_parts(state: AgentState) -> AgentState:
    """
        Busca el repuesto en MongoDB usando el código identificado.
        Muestra los resultados por pantalla de forma detallada.
    """
    codigo = state.get("codigo_repuesto")
    try:
        # Buscar en MongoDB por id_repuesto
        resultados = list(collection.find({"id_repuesto": codigo}))
        internos = [r for r in resultados if r.get('proveedor_tipo') == 'INTERNAL']
        print(f"✅ Se encontraron {len(internos)} opción(es) para el repuesto {codigo}\n")

    except Exception as e:
        print(f"❌ Error al buscar en la base de datos: {e}")


# Generación del grafo

In [8]:
memory = MemorySaver()

#Defino el grafo
graph_builder = StateGraph(AgentState)

#Agrego el nodo al grafo
graph_builder.add_node("llm_node", nodo_llm)
graph_builder.add_node("validation", classify_request)
graph_builder.add_node("set_val_message", set_val_message)
graph_builder.add_node("search_internal_parts", search_internal_parts)

#Conecto los nodos (Solo uno por ahora)
graph_builder.add_edge(START, "validation")
graph_builder.add_conditional_edges(
    "validation",
    route_classification,
    {
        "continue": "llm_node",
        "end": "set_val_message"
    }
)
graph_builder.add_edge("set_val_message", END)
graph_builder.add_conditional_edges(
    "llm_node",
    tiene_info_suficiente,
    {
        "continue": "search_internal_parts",  
        "request_more_info": "llm_node",
        "end": END
    }
)
graph_builder.add_edge("search_internal_parts", END)

graph = graph_builder.compile(
    checkpointer=memory, 
    interrupt_after=["llm_node"]
)

In [9]:
#Defino la funcion del agente
def iniciar_agente(mensaje_usuario: str):
    config = {"configurable": {"thread_id": "1"}}
    
    # Estado inicial con todos los campos
    estado_inicial = {
        "messages": [HumanMessage(content=mensaje_usuario)],
        "codigo_repuesto": None,
        "info_completa": False,
        "validation_result": ValidationRequest()
    }
    
    result = graph.invoke(estado_inicial, config)
    
    # Loop de conversación
    while True:
        # Mostrar respuesta del agente
        print(f"\n🤖 Agente: {result['messages'][-1].content}")
        
        # Verificar si llegamos al final (tenemos el código)
        if result.get("info_completa", False):
            print(f"\n{'='*60}")
            print(f"📋 Información recopilada:")
            print(f"   Código del repuesto: {result['codigo_repuesto']}")
            print(f"{'='*60}")
            result = graph.invoke(None, config)
            print("Resultados mostrados")
            break
        
        # Pedir nuevo mensaje al usuario
        nuevo_mensaje = input("\n👤 Tú: ")
        
        if nuevo_mensaje.lower() in ["salir", "exit", "quit"]:
            print("\n👋 Conversación terminada")
            break
        
        # Continuar la conversación
        nuevo_estado = {"messages": [HumanMessage(content=nuevo_mensaje)]}
        result = graph.invoke(nuevo_estado, config)
    
    return result

## Ejemplos de uso del agente

**Ejemplos de códigos de repuestos en la base de datos:**
- R-0001: Rodamiento rígido de bolas 6204 2RS
- R-0002: Filtro de aceite motor diésel
- R-0005: Bomba centrífuga 3 HP
- R-0010: Manómetro glicerina 0-16 bar

**Categorías disponibles:**
- RODAMIENTO, FILTRO, CORREA, SENSOR, BOMBA, ELECTRICO, NEUMATICA, MECANICO, INSTRUMENTO


In [10]:
#main
print("="*60)
print("🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS")
print("="*60)
print("\nBienvenido al sistema de búsqueda de repuestos.")
print("El agente te ayudará a encontrar el repuesto que necesitas.")
print("\nPuedes escribir 'salir' en cualquier momento para terminar.\n")
print("-"*60)

mensaje_usuario = input("\n👤 Tú: ")
resultado = iniciar_agente(mensaje_usuario)

#Repuesto con el codigo R-0001


🔧 SISTEMA DE BÚSQUEDA DE REPUESTOS

Bienvenido al sistema de búsqueda de repuestos.
El agente te ayudará a encontrar el repuesto que necesitas.

Puedes escribir 'salir' en cualquier momento para terminar.

------------------------------------------------------------
Dentro de route: False

🤖 Agente: La consulta no es un pedido de repuestos. Por favor, proporcione detalles específicos sobre los repuestos o piezas que necesita para que podamos procesar su solicitud.

👋 Conversación terminada
